# Bioinformatics Project - Computational Drug Discovery - Influenza virus A matrix protein M2  
Michael Bahchevanov  
***

## Feature Engineering 🚀  
This notebook will be looking into calculating the **molecular descriptors** and **fingerprints** via the *Morgan Fingerprint* which is a mathematically encoded *2048* bit vector. **Molecular fingerprints** are a way of encoding the structure of a molecule. They are most often represented in binary digits that represent the presence or absence of a particular substructure in the molecule. The molecular fingerprints are an essential part of the ***QSAR*** workflow and are a part of the fundamental cheminformatics tools for **virtual screening** and **mapping chemical space**. In particular, **substructure fingerprints** perform best for small molecules such as drugs (our target).  <br>[*QSAR-derived affinity fingerprints* - <a href="https://jcheminf.biomedcentral.com/track/pdf/10.1186/s13321-020-00444-5.pdf">Reference</a>, *Study on molecular fingerprints for drugs, biomolecules, and the metabolome* - <a href="https://jcheminf.biomedcentral.com/articles/10.1186/s13321-020-00445-4">Reference</a>]

We have already analyzed our data using the *EC50* measurement of concentration. This gave us a general overview of the bioactive binding compounds. Now our goal is to calculate the **molecular fingerprints**. As the *ChEMBL* database is well-maintained and somewhat standardized, for calculating the fingerprints we will be considering using also compounds with different measurements of *activation/inhibition*. Those would be namely:  
* EC$_{50}$ - Half maximal effective concentration  
* IC$_{50}$ - How much of a particular inhibitory substance (e.g. drug) is needed to inhibit, in vitro, a given biological process or biological component by 50%  
* K$_{i}$ - The concentration required to produce half maximum inhibition  
* K$_{d}$ - The equilibrium dissociation constant, which is used to evaluate and rank order strengths of interactions between two molecules, measurement for the binding affinity of a compound

There are two particular machine learning algorithms that have repeatedly shown consistent results (*AUCs* of around 0.7-0.8) - **Random Forests** and **Naive Bayes**. Although, for this preparation we will also be looking into **Random Matrix Theory** which is an algorithm framework (developed by <a href="https://www.alpha-lee.com/">Dr. Alpha Lee's group</a>) that promises high results.  <br>[*RMT, Alpha Lee Paper* - <a href="https://www.pnas.org/content/pnas/113/48/13564.full.pdf">Reference</a>]  <br> <br>
*Note - AUC stands for the area under the ROC (Receiver Operating Characteristic) curve. It is a graph showing the performance of a classification model at all classification thresholds and plots two parameters - True Positive and False Positive*  <br>
[<a href="https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-classification-in-python/">Reference</a>]

### 1. Data Collection and Cleaning  
We will need 2 datasets:  
* a set of drugs that **bind** to a particular target (in our case the *Influenza virus A matrix protein M2*)  
* a set of drugs that **do not bind** to the particular target (a **decoy set**)  
For our **decoy set** we will be using the receptor used in <a href="https://www.pnas.org/content/pnas/113/48/13564.full.pdf">*Alpha Lee's Paper*</a> - *5HT1A*. The binding affinity to this receptor is irrelevant as we assume that the binding affinity to the *Influenza virus A matrix protein M2* is negligible (a reasonable assumption due to them being non-related). As we will be looking only for binder molecules, we will be selecting only the molecules that have a binding affinity of less that **1000nM**.  <br>
[<a href="https://towardsdatascience.com/random-matrix-theory-the-best-classifier-for-prediction-of-drug-binding-f82613fb48ed"> Reference</a>]

#### 1.1 Importing Libraries and Tooling 🔨  
We will be using *rdkit* for cheminformatics, *numpy* for computing, and *pandas* for data wrangling and loading. We will also need the *chembl_web_client* for querying the database.

In [3]:
import rdkit
from rdkit import Chem
from rdkit.Chem import PandasTools

import numpy as np
import pandas as pd

from chembl_webresource_client.new_client import new_client

In [4]:
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#### 1.2 Collecting the 5HT1A data from ChEMBL

In [5]:
target = new_client.target
query = target.search('5HT1A')
targets = pd.DataFrame.from_dict(query)
print(targets.shape)
targets

(9, 9)


,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,"[{'xref_id': 'P19327', 'xref_name': None, 'xre...",Rattus norvegicus,Serotonin 1a (5-HT1a) receptor,24.0,False,CHEMBL273,"[{'accession': 'P19327', 'component_descriptio...",SINGLE PROTEIN,10116
1,[],Rattus norvegicus,Serotonin (5-HT) receptor,24.0,False,CHEMBL2094123,"[{'accession': 'P35563', 'component_descriptio...",PROTEIN FAMILY,10116
2,[],Rattus norvegicus,Serotonin 1 (5-HT1) receptor,24.0,False,CHEMBL2095159,"[{'accession': 'P28564', 'component_descriptio...",PROTEIN FAMILY,10116
3,[],Rattus norvegicus,Serotonin 1a (5-HT1a) receptor/Adrenergic rece...,24.0,False,CHEMBL2097171,"[{'accession': 'P15823', 'component_descriptio...",SELECTIVITY GROUP,10116
4,"[{'xref_id': 'P08908', 'xref_name': None, 'xre...",Homo sapiens,Serotonin 1a (5-HT1a) receptor,15.0,False,CHEMBL214,"[{'accession': 'P08908', 'component_descriptio...",SINGLE PROTEIN,9606
5,"[{'xref_id': 'NBK23263', 'xref_name': '5-HT1A ...",Mus musculus,Serotonin 1a (5-HT1a) receptor,15.0,False,CHEMBL3737,"[{'accession': 'Q64264', 'component_descriptio...",SINGLE PROTEIN,10090
6,[],Homo sapiens,Dopamine D2 receptor and serotonin 1a receptor,13.0,False,CHEMBL2111460,"[{'accession': 'P14416', 'component_descriptio...",SELECTIVITY GROUP,9606
7,[],Homo sapiens,Serotonin 1 (5-HT1) receptor,8.0,False,CHEMBL4524122,"[{'accession': 'P30939', 'component_descriptio...",PROTEIN FAMILY,9606
8,[],Homo sapiens,Serotonin (5-HT) receptor,4.0,False,CHEMBL2096904,"[{'accession': 'P30939', 'component_descriptio...",PROTEIN FAMILY,9606


We will be getting the **Single Protein** from index 4 - which denotes the receptor in humans.

In [6]:
selected_target = targets.target_chembl_id[4]
selected_target

'CHEMBL214'

In [7]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target)
df = pd.DataFrame.from_dict(res)
print(df.shape)
df.head(100)

(9899, 45)


,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,bao_label,canonical_smiles,data_validity_comment,data_validity_description,document_chembl_id,document_journal,document_year,ligand_efficiency,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,potential_duplicate,qudt_units,record_id,relation,src_id,standard_flag,standard_relation,standard_text_value,standard_type,standard_units,standard_upper_value,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,32187,[],CHEMBL616136,Affinity for 5-hydroxytryptamine 1A receptor s...,B,None,None,BAO_0000192,BAO_0000357,single protein format,COc1ccc(NC(=O)c2ccc(-c3ccc(-c4noc(C)n4)cc3C)cc...,None,None,CHEMBL1150041,Bioorg. Med. Chem. Lett.,1997.0,"{'bei': '14.35', 'le': '0.26', 'lle': '2.11', ...",CHEMBL15928,GR-127935,CHEMBL15928,7.14,False,http://www.openphacts.org/units/Nanomolar,203462,=,1,True,=,None,Ki,nM,None,72.0,CHEMBL214,Homo sapiens,Serotonin 1a (5-HT1a) receptor,9606,None,None,Ki,nM,UO_0000065,None,72.0
1,None,32515,[],CHEMBL849639,Ki ratio measured as the Ki values of dopamine...,B,None,None,BAO_0000179,BAO_0000357,single protein format,c1ccc2c(c1)CCC2CN1CCN(c2cccc3c2OCCO3)CC1,None,None,CHEMBL1128459,J. Med. Chem.,1995.0,None,CHEMBL131011,None,CHEMBL131011,None,False,None,250186,=,1,False,=,None,Ki ratio,None,None,43.6,CHEMBL214,Homo sapiens,Serotonin 1a (5-HT1a) receptor,9606,None,None,Ki ratio,None,None,None,43.6
2,None,33363,[],CHEMBL616136,Affinity for 5-hydroxytryptamine 1A receptor s...,B,None,None,BAO_0000192,BAO_0000357,single protein format,Cc1ccccc1N1CCN(C(=O)COc2ccc3[nH]cc(CCN)c3c2)CC...,None,None,CHEMBL1150041,Bioorg. Med. Chem. Lett.,1997.0,"{'bei': '20.63', 'le': '0.38', 'lle': '5.39', ...",CHEMBL321818,None,CHEMBL120055,8.10,False,http://www.openphacts.org/units/Nanomolar,203458,=,1,True,=,None,Ki,nM,None,8.0,CHEMBL214,Homo sapiens,Serotonin 1a (5-HT1a) receptor,9606,None,None,Ki,nM,UO_0000065,None,8.0
3,None,34629,[],CHEMBL616136,Affinity for 5-hydroxytryptamine 1A receptor s...,B,None,None,BAO_0000192,BAO_0000357,single protein format,COc1ccc(NC(=O)N2CCN(c3ccccc3)CC2)cc1N1CCN(C)CC...,None,None,CHEMBL1150041,Bioorg. Med. Chem. Lett.,1997.0,"{'bei': '15.50', 'le': '0.29', 'lle': '3.55', ...",CHEMBL325131,None,CHEMBL1183944,6.35,False,http://www.openphacts.org/units/Nanomolar,203465,=,1,True,=,None,Ki,nM,None,450.0,CHEMBL214,Homo sapiens,Serotonin 1a (5-HT1a) receptor,9606,None,None,Ki,nM,UO_0000065,None,450.0
4,None,35741,[],CHEMBL615755,Binding affinity of compound to human 5-hydrox...,B,None,None,BAO_0000192,BAO_0000357,single protein format,Nc1cccc(-c2ccc(CCN3CCN(c4cccc5cccnc45)CC3)cc2)n1,None,None,CHEMBL1131962,Bioorg. Med. Chem. Lett.,1999.0,"{'bei': '20.81', 'le': '0.38', 'lle': '4.28', ...",CHEMBL87187,None,CHEMBL87187,8.52,False,http://www.openphacts.org/units/Nanomolar,155825,=,1,True,=,None,Ki,nM,None,3.0,CHEMBL214,Homo sapiens,Serotonin 1a (5-HT1a) receptor,9606,None,None,Ki,nM,UO_0000065,None,3.0
5,None,35839,[],CHEMBL616136,Affinity for 5-hydroxytryptamine 1A receptor s...,B,None,None,BAO_0000192,BAO_0000357,single protein format,Cc1ccccc1N1CCN(C(=O)Oc2ccc(Cl)c(N3CCN(C)CC3)c2...,None,None,CHEMBL1150041,Bioorg. Med. Chem. Lett.,1997.0,None,CHEMBL107772,None,CHEMBL1179973,None,False,http://www.openphacts.org/units/Nanomolar,203463,>,1,True,>,None,Ki,nM,None,1000.0,CHEMBL214,Homo sapiens,Serotonin 1a (5-HT1a) receptor,9606,None,None,Ki,nM,UO_0000065,None,1000.0
6,None,38320,[],CHEMBL616136,Affinity for 5-hydroxytryptamine 1A receptor s...,B,None,None,BAO_0000192,BAO_0000357,single protein format,COc1ccc(NC(=O)N2CCN(c3ccc(C)cc3C)CC2)cc1N1CCN(...,None,None,CHEMBL1150041,Bioorg. Med. Chem. Lett.,1997.0,"{'bei': '14.27', 'le': '0.27', 'lle': '2.82', ...",CHEMBL323986,None,CHEMBL1183904,6.24,False,http://www.openphacts.org/units/Nan

In [8]:
df.to_csv('./data/5HT1A_01_bioactivity_data_raw.csv', index=False)

#### 1.3 Data Cleaning  
Now we will be creating a function that would normalize the values, drop null values, select only the entries who are recorded in **nM**, and finally drop the entries who do not have their **SMILES** notation, as we will be calculating the **Morgan Fingerprint** based on that notation.

In [9]:
def load_dataset(file, drop_non_binders=True):
    df = pd.read_csv(file, encoding = 'ISO-8859-1')
    
    # Converting strings to floats and setting all non-numbers to NaN
    df['standard_value'] = pd.to_numeric(df['standard_value'], errors='coerce')
    
    # Drop NaNs in the affinity column
    df.dropna(subset=['standard_value'], inplace=True)
    df.reset_index(inplace=True)
    df.drop('index', axis=1, inplace=True)
    
    # Filtering for only activities recorded in nanomolar (nM) affinity
    df = df[df['standard_units'] == 'nM']
    
    # Dropping molecules that do not have a SMILES notation recorded
    df.dropna(subset=['canonical_smiles'])
    
    # Considering only the binders (affinity < 1000nM)
    if drop_non_binders:
        df = df[df['standard_value'] < 1000]
    
    # Dropping duplicate molecules
    df.drop_duplicates(subset=['canonical_smiles'], keep='first', inplace=True)
    
    return df

In [12]:
df_train_test = load_dataset('./data/influenza_virus_A_matrix_M2_protein_01_bioactivity_data_raw.csv')
df_decoy = load_dataset('./data/5HT1A_01_bioactivity_data_raw.csv', drop_non_binders=False)

We are naming the bioactivity dataset for the *M2 matrix protein* as `train_test`. This is due to the fact that these are already compounds that have proven *inhibitory/activatory* response to the target protein. While our goal is to find compounds based on the **molecular fingerprint**, this data set is going to serve as our *validation* dataset.

In [19]:
df_train_test

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,bao_label,canonical_smiles,data_validity_comment,data_validity_description,document_chembl_id,document_journal,document_year,ligand_efficiency,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,potential_duplicate,qudt_units,record_id,relation,src_id,standard_flag,standard_relation,standard_text_value,standard_type,standard_units,standard_upper_value,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
1,NaN,18753173,[],CHEMBL4264733,Inhibition of wild type Influenza A virus (A/c...,B,NaN,NaN,BAO_0000188,BAO_0000019,assay format,N#CCCN(Cc1cccs1)S(=O)(=O)c1cccc(F)c1,NaN,NaN,CHEMBL4261607,Eur J Med Chem,2018,"{'bei': '20.95', 'le': '0.44', 'lle': '3.81', ...",CHEMBL4277167,NaN,CHEMBL4277167,6.80,False,http://www.openphacts.org/units/Nanomolar,3115887,=,1,True,=,NaN,EC50,nM,NaN,160.0,CHEMBL2052,Influenza A virus (A/Udorn/307/1972(H3N2)),Influenza virus A matrix protein M2,381517,NaN,NaN,EC50,uM,UO_0000065,NaN,0.160
2,NaN,18753174,[],CHEMBL4264733,Inhibition of wild type Influenza A virus (A/c...,B,NaN,NaN,BAO_0000188,BAO_0000019,assay format,N#CCCN(Cc1cccs1)S(=O)(=O)c1ccc(F)cc1,NaN,NaN,CHEMBL4261607,Eur J Med Chem,2018,"{'bei': '21.13', 'le': '0.45', 'lle': '3.86', ...",CHEMBL4284419,NaN,CHEMBL4284419,6.85,False,http://www.openphacts.org/units/Nanomolar,3115888,=,1,True,=,NaN,EC50,nM,NaN,140.0,CHEMBL2052,Influenza A virus (A/Udorn/307/1972(H3N2)),Influenza virus A matrix protein M2,381517,NaN,NaN,EC50,uM,UO_0000065,NaN,0.140
3,NaN,18753175,[],CHEMBL4264733,Inhibition of wild type Influenza A virus (A/c...,B,NaN,NaN,BAO_0000188,BAO_0000019,assay format,N#CCCN(Cc1cccs1)S(=O)(=O)c1ccc(Cl)cc1,NaN,NaN,CHEMBL4261607,Eur J Med Chem,2018,"{'bei': '18.11', 'le': '0.40', 'lle': '2.66', ...",CHEMBL4292316,NaN,CHEMBL4292316,6.17,False,http://www.openphacts.org/units/Nanomolar,3115889,=,1,True,=,NaN,EC50,nM,NaN,670.0,CHEMBL2052,Influenza A virus (A/Udorn/307/1972(H3N2)),Influenza virus A matrix protein M2,381517,NaN,NaN,EC50,uM,UO_0000065,NaN,0.670
4,NaN,18753176,[],CHEMBL4264733,Inhibition of wild type Influenza A virus (A/c...,B,NaN,NaN,BAO_0000188,BAO_0000019,assay format,N#CCCN(Cc1cccs1)S(=O)(=O)c1ccc(Br)cc1,NaN,NaN,CHEMBL4261607,Eur J Med Chem,2018,"{'bei': '16.55', 'le': '0.41', 'lle': '2.76', ...",CHEMBL4294990,NaN,CHEMBL4294990,6.38,False,http://www.openphacts.org/units/Nanomolar,3115890,=,1,True,=,NaN,EC50,nM,NaN,420.0,CHEMBL2052,Influenza A virus (A/Udorn/307/1972(H3N2)),Influenza virus A matrix protein M2,381517,NaN,NaN,EC50,uM,UO_0000065,NaN,0.420
5,NaN,18753177,[],CHEMBL4264733,Inhibition of wild type Influenza A virus (A/c...,B,NaN,NaN,BAO_0000188,BAO_0000019,assay format,Cc1ccc(S(=O)(=O)N(CCC#N)Cc2cccs2)cc1,NaN,NaN,CHEMBL4261607,Eur J Med Chem,2018,"{'bei': '19.51', 'le': '0.41', 'lle': '3.09', ...",CHEMBL4286741,NaN,CHEMBL4286741,6.25,False,http://www.openphacts.org/units/Nanomolar,3115891,=,1,True,=,NaN,EC50,nM,NaN,560.0,CHEMBL2052,Influenza A virus (A/Udorn/307/1972(H3N2)),Influenza virus A matrix protein M2,381517,NaN,NaN,EC50,uM,UO_0000065,NaN,0.560
7,NaN,18753179,[],CHEMBL4264733,Inhibition of wild type Influenza A virus (A/c...,B,NaN,NaN,BAO_0000188,BAO_0000019,assay format,N#CCCN(Cc1cccs1)S(=O)(=O)c1ccc(C#N)cc1,NaN,NaN,CHEMBL4261607,Eur J Med Chem,2018,"{'bei': '24.43', 'le': '0.50', 'lle': '5.38', ...",CHEMBL4277990,NaN,CHEMBL4277990,8.10,False,http://www.openphacts.org/units/Nanomolar,3115893,=,1,True,=,NaN,EC50,nM,NaN,8.0,CHEMBL2052,Influenza A virus (A/Udorn/307/1972(H3N2)),Influenza virus A matrix protein M2,381517,NaN,NaN,EC50,uM,UO_0000065,NaN,0.008
13,NaN,18753185,[],CHEMBL4264733,Inhibition of wild type Influenza A virus (A/c...,B,NaN,NaN,BAO_0000188,BAO_0000019,assay format,CCCN(Cc1cccs1)S(=O)(=O)c1ccc(Br)cc1,NaN,NaN,CHEMBL4261607,Eur J Med C

As for the **decoy**, it is assumed that it by no means has any relation to the responses in our target protein.

In [15]:
df_decoy.sample(100)

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,bao_label,canonical_smiles,data_validity_comment,data_validity_description,document_chembl_id,document_journal,document_year,ligand_efficiency,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,potential_duplicate,qudt_units,record_id,relation,src_id,standard_flag,standard_relation,standard_text_value,standard_type,standard_units,standard_upper_value,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
7981,NaN,18585930,[],CHEMBL4223417,Displacement of [3H]-Way100635 from 5HT1A rece...,B,NaN,NaN,BAO_0000192,BAO_0000219,cell-based format,CCCCCN1CCC(c2cccc(O)c2)C1,NaN,NaN,CHEMBL4219212,Bioorg Med Chem Lett,2018.0,"{'bei': '27.80', 'le': '0.52', 'lle': '3.12', ...",CHEMBL4228054,NaN,CHEMBL4228054,6.49,False,http://www.openphacts.org/units/Nanomolar,3086464,=,1,True,=,NaN,Ki,nM,NaN,3.260000e+02,CHEMBL214,Homo sapiens,Serotonin 1a (5-HT1a) receptor,9606,NaN,NaN,Ki,nM,UO_0000065,NaN,326.000
5026,NaN,3598408,[],CHEMBL1293141,Antagonist activity at human 5-HT1A receptor e...,F,NaN,NaN,BAO_0000192,BAO_0000219,cell-based format,Cc1ccc2c(C3CCN(CCc4c(C)ccc5c4ccc(=O)n5C)CC3)cc...,NaN,NaN,CHEMBL1287678,Bioorg. Med. Chem. Lett.,2010.0,NaN,CHEMBL1289721,NaN,CHEMBL1289721,8.90,False,http://www.openphacts.org/units/Nanomolar,975102,=,1,True,=,NaN,Ki,nM,NaN,1.259000e+00,CHEMBL214,Homo sapiens,Serotonin 1a (5-HT1a) receptor,9606,NaN,NaN,pKi,NaN,UO_0000065,NaN,8.900
6385,NaN,14999728,[],CHEMBL3364795,Displacement of [3H]-8-OH-DPAT from human 5-HT...,B,NaN,NaN,BAO_0000192,BAO_0000219,cell-based format,O=C1Cc2ccccc2N1CCCCCN1CC[C@H]2CCCC[C@@H]2C1,NaN,NaN,CHEMBL3351619,J. Med. Chem.,2014.0,"{'bei': '18.53', 'le': '0.34', 'lle': '2.05', ...",CHEMBL3321796,NaN,CHEMBL3321796,6.31,False,http://www.openphacts.org/units/Nanomolar,2248874,=,1,True,=,NaN,Ki,nM,NaN,4.910000e+02,CHEMBL214,Homo sapiens,Serotonin 1a (5-HT1a) receptor,9606,NaN,NaN,Ki,nM,UO_0000065,NaN,491.000
4855,NaN,3508334,[],CHEMBL1259255,Intrinsic activity at human 5-HT1A receptor ex...,F,NaN,NaN,BAO_0000188,BAO_0000219,cell-based format,Cc1ccc(CNCC2(F)CCN(C(=O)c3sccc3C)CC2)nc1,NaN,NaN,CHEMBL1255227,J. Med. Chem.,2010.0,NaN,CHEMBL1259122,NaN,CHEMBL1259122,7.46,False,http://www.openphacts.org/units/Nanomolar,964168,=,1,True,=,NaN,EC50,nM,NaN,3.500000e+01,CHEMBL214,Homo sapiens,Serotonin 1a (5-HT1a) receptor,9606,NaN,NaN,EC50,nM,UO_0000065,NaN,35.000
7284,NaN,16819777,[],CHEMBL3872757,Displacement of [3H]8-OH-DPAT from recombinant...,B,NaN,NaN,BAO_0000192,BAO_0000219,cell-based format,CN(C)C(=O)N[C@H]1CC[C@H](CCN2CCN(c3nsc4ccccc34...,NaN,NaN,CHEMBL3872208,Eur J Med Chem,2016.0,"{'bei': '21.46', 'le': '0.42', 'lle': '5.28', ...",CHEMBL3960785,NaN,CHEMBL3960785,8.92,False,http://www.openphacts.org/units/Nanomolar,2812749,=,1,True,=,NaN,Ki,nM,NaN,1.200000e+00,CHEMBL214,Homo sapiens,Serotonin 1a (5-HT1a) receptor,9606,NaN,NaN,Ki,nM,UO_0000065,NaN,1.200
6287,NaN,14576192,[],CHEMBL3241203,Displacement of [3H]-8-OH-DPAT from human 5-HT...,B,NaN,NaN,BAO_0000192,BAO_0000219,cell-based format,COc1ccccc1N1CCN(CCCCCC(=O)N2CCC[C@H]2C(N)=O)CC1,NaN,NaN,CHEMBL3232956,Eur. J. Med. Chem.,2014.0,"{'bei': '18.28', 'le': '0.35', 'lle': '5.51', ...",CHEMBL3235731,NaN,CHEMBL3235731,7.36,False,http://www.openphacts.org/units/Nanomolar,2084690,=,1,True,=,NaN,Ki,nM,NaN,4.400000e+01,CHEMBL214,Homo sapiens,Serotonin 1a (5-HT1a) receptor,9606,NaN,NaN,Ki,nM,UO_0000065,NaN,44.000
2364,NaN,1602044,[],CHEMBL884790,Binding affinity towards 5-HT1A receptor using...,B,NaN,NaN,BAO_0000192,BAO_0000019,assay format,O=C(NCCCCN1CCN(c2cccc(Cl)c2Cl)CC1)c1ccc(F)cc1,NaN,NaN,CHEMBL1143473,Bioorg. Med. Chem. Lett.,2005.0,"{'bei': '17.02', 'le': '0.35', 'lle': '2.76', ...",CHEMBL196744,NaN,CHEMBL196744,7.22,False,http://www.openphacts.org/units/Nanomolar,400662,=,1,True,=,N

#### 1.4 Feature selection  
Finally, we are selecting only the relevant columns. We will not be looking into the affinity columns as we already know that our *train_test* dataset contains only **binders** (< 1000nM).

In [27]:
smiles_train_test = df_train_test['canonical_smiles']
smiles_train_test = smiles_train_test.reset_index()['canonical_smiles']
affinity_train_test = df_train_test['standard_value']

binding_treshold = 1000
affinity_train_test = affinity_train_test.apply(lambda x: 0 if x< 1000 else 1)

smiles_decoy = df_decoy['canonical_smiles']
smiles_decoy = smiles_decoy.reset_index()['canonical_smiles']
affinity_decoy = pd.Series([0 for i in range(len(smiles_decoy))])

### 2. Generating the Molecular Fingerprint  
We are looking to convert the **molecular structure** into a **mathematical object**, to be able to apply machine learning algorithms on it. We will be encoding the **molecular structure** into a ***Morgan bit vector*** of length 2048. Each entry would be either the presence or absence of a particular chemical substructure.  
<img src="./assets/Morgan_Fingerprint.png"/>

In [40]:
def generate_morgan_matrix(smiles):
    morgan_matrix = np.zeros((1, 2048))
    l = len(smiles)
    
    # For each compound, get the structure, convert to Morgan fingerprint, and add to the morgan_matrix
    for i in range(l):
        try:
            compound = Chem.MolFromSmiles(smiles[i])
            fingerprint = Chem.AllChem.GetMorganFingerprintAsBitVect(compound, 2, nBits=2048)
            fingerprint = fingerprint.ToBitString()
            row = np.array([int(x) for x in list(fingerprint)])
            morgan_matrix = np.row_stack((morgan_matrix, row))
            
            # Progress checker
            if i % 500 == 0:
                percentage = np.round(100*(i/l), 1)
                print(f'{percentage}% done')
        except:
            print(f'problem at index: {i}')
    
    # Deleting the first row of zeros
    morgan_matrix = np.delete(morgan_matrix, 0, axis=0)
    
    print('\n')
    print(f'Morgan Matrix dimensions: {morgan_matrix.shape}')
    return morgan_matrix

In [49]:
morgan_matrix_train_test = generate_morgan_matrix(smiles_train_test)
morgan_matrix_decoy = generate_morgan_matrix(smiles_decoy)

0.0% done


Morgan Matrix dimensions: (14, 2048)
0.0% done
9.6% done
19.2% done
problem at index: 1392
28.8% done
38.5% done
48.1% done
57.7% done
67.3% done
76.9% done
86.5% done
96.1% done


Morgan Matrix dimensions: (5200, 2048)


The matrixes are of dimensions ***n x m*** where **n** denotes the number of molecules and **m** denotes the number of bit features (which are expected to be *2048* as the **Morgan bit vector** is of length 2048).  <br> <br>
Now, we will be saving the results of the **Morgan matrix** generation to a *.csv* file.

In [51]:
pd.DataFrame(morgan_matrix_train_test).to_csv('./data/influenza_virus_A_matrix_M2_protein_06_morgan_matrix.csv', index=False)
pd.DataFrame(morgan_matrix_decoy).to_csv('./data/5HT1A_02_morgan_matrix.csv', index=False)

*** 
### Overview
In this notebook we have mathematically encoded the **molecular structure** of our target protein, *Influenza virus A matrix M2 protein*. We have also collected and cleaned a **decoy** set that would serve for modelling later. The **decoy** set consists of the *5HT1A* receptor in humans for which we have also applied the same encoding technique as for our target protein. In the following notebook we will be performing a **Principal Component Analysis** in which we will be looking for the most important features or combination of features in the training set.